In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd

In [4]:
train_data = pd.read_json('/content/train.jsonl',lines=True)
validate_data = pd.read_json('/content/validation.jsonl',lines=True)

In [5]:
#filter only phrase tags for Q&A task
train_data['tags']=train_data['tags'].apply(lambda x:x[0])
train_data=train_data[train_data['tags']=='phrase'].reset_index(drop=True)

#filter only phrase tags for Q&A task
validate_data['tags']=validate_data['tags'].apply(lambda x:x[0])
validate_data=validate_data[validate_data['tags']=='phrase'].reset_index(drop=True)

In [6]:
# data processing to get context and spoiler start position
def create_df(dataset):

    col1=[]
    col2=[]

    for idx in range(len(dataset)):
        concatlen=0
        line_occuring=dataset['spoilerPositions'][idx][0][0][0]

        for i in range(line_occuring):
            concatlen+=len(dataset['targetParagraphs'][idx][i])


        starting_position=concatlen+dataset['spoilerPositions'][idx][0][0][1]

        col1.append({'answer_start': [starting_position], 'text': dataset['spoiler'][idx]})


        l=''
        for line in dataset['targetParagraphs'][idx]:
            l+=line
        
        col2.append(l)

    df = pd.DataFrame(list(zip(col1, col2)),
    columns =['answers', 'context'])
    df['question']=dataset['postText'].apply(lambda x: x[0])
    df['id']=dataset['postId']

    json_data=[]
    for index, row in df.iterrows():
        json_data.append({
            'question': row['question'],
            'context': row['context'],
            'answers': row['answers'],
            'id': row['id']
        })

    return json_data

In [7]:
import json

train_json=create_df(train_data)
validate_json=create_df(validate_data)

with open('train_json.json', 'w') as f:
    json.dump(train_json, f)

with open('validate_json.json', 'w') as f:
    json.dump(validate_json, f)

In [8]:
from datasets import load_dataset, DatasetDict, load_metric

In [9]:
# Load the JSON file as a Dataset object
train_dataset = load_dataset('json', data_files='train_json.json')
validate_dataset = load_dataset('json', data_files='validate_json.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-983fcf4a899a01b3/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5ed83ed70ad5fde6/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# Create a DatasetDict object containing the Dataset
train_dict = DatasetDict({'train': train_dataset['train']})
valid_dict = DatasetDict({'valid': validate_dataset['train']})

In [11]:
from transformers import AutoTokenizer
#model_checkpoint = "distilbert-base-uncased"
model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)

In [12]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [14]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [13]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [21]:
tokenized_train = train_dict.map(prepare_train_features, batched=True, remove_columns=train_dict["train"].column_names)
tokenized_valid = valid_dict.map(prepare_validation_features, batched=True, remove_columns=valid_dict["valid"].column_names)

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

# TO DO


In [24]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model)

OSError: ignored

In [17]:
batch_size=16

In [18]:
model_name = "bert-base-uncased"
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [19]:
from transformers import default_data_collator

data_collator = default_data_collator

In [20]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_valid["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/content/bert-base-uncased-finetuned-squad is already a clone of https://huggingface.co/aravind168/bert-base-uncased-finetuned-squad. Make sure you pull the latest changes with `repo.git_pull()`.


In [23]:
trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,1.936600,No log


TrainOutput(global_step=600, training_loss=1.8179331588745118, metrics={'train_runtime': 701.867, 'train_samples_per_second': 13.665, 'train_steps_per_second': 0.855, 'total_flos': 1879572895391232.0, 'train_loss': 1.8179331588745118, 'epoch': 3.0})

In [24]:
trainer.save_model("test-squad-trained")

Upload file pytorch_model.bin:   0%|          | 1.00/415M [00:00<?, ?B/s]

Upload file runs/Apr19_19-57-35_93a65849b10e/events.out.tfevents.1681934301.93a65849b10e.1087.0:   0%|        …

To https://huggingface.co/aravind168/bert-base-uncased-finetuned-squad
   5cb8073..24fc8a0  main -> main

   5cb8073..24fc8a0  main -> main

To https://huggingface.co/aravind168/bert-base-uncased-finetuned-squad
   24fc8a0..bc59f5d  main -> main

   24fc8a0..bc59f5d  main -> main



In [26]:
from tqdm.auto import tqdm
import numpy as np
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not True:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [27]:
raw_predictions = trainer.predict(tokenized_valid['valid'])

In [29]:
final_predictions = postprocess_qa_predictions(valid_dict['valid'], tokenized_valid['valid'], raw_predictions.predictions)

Post-processing 335 example predictions split into 768 features.


  0%|          | 0/335 [00:00<?, ?it/s]

In [30]:
#to see output. comment if not needed.

final_predictions

OrderedDict([('857559227384160256',
              ', since the goal is to get you back into the salon. When in doubt, ask at'),
             ('388301310635671553',
              ', not by the ethanol itself, but by ethanol’s first metabolite, acetaldehyde. Ethanol is metabolized into acetal'),
             ('553676574290354177', 'Recipe created by Art SmithSer'),
             ('4kt69b',
              'lettuce and bowls of soup and Mr. Obama is holding up'),
             ('4wagdb', ', according to both the'),
             ('1100188040002123_1123063127714614',
              ', until it dissipated rapidly into the cold air. The gas could be clearly seen against the dark-colored ea'),
             ('473452702002065409',
              ', which will be published in full in the'),
             ('170016066732231_174009032999601',
              ', and the two obvious paths are to a) Re-release a tweaked version of iOS 9.3.2 or b'),
             ('474005518713769985', ', The'),
             ('4k

In [31]:
metric = load_metric("squad_v2")
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in valid_dict['valid']]
metric.compute(predictions=formatted_predictions, references=references)

<ipython-input-31-3bdf1938f8b2>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")


{'exact': 0.0,
 'f1': 3.1715596833699937,
 'total': 335,
 'HasAns_exact': 0.0,
 'HasAns_f1': 3.1715596833699937,
 'HasAns_total': 335,
 'best_exact': 0.0,
 'best_exact_thresh': 0.0,
 'best_f1': 3.1715596833699937,
 'best_f1_thresh': 0.0}

In [33]:
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 5.0 MB/s eta 0:00:00


In [34]:
metric = load_metric("sacrebleu")
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in valid_dict['valid']]
# metric.compute(predictions=formatted_predictions, references=references)
metric.compute(predictions=formatted_predictions, references=references, smooth_method='floor', smooth_value=0.2)

{'score': 0.0,
 'counts': [0, 0, 0, 0],
 'totals': [11007, 10672, 10337, 10002],
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 11007,
 'ref_len': 335}